In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-as92r_rn/unsloth_f1cd0ece43994f3289fea25ea0d15971
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-as92r_rn/unsloth_f1cd0ece43994f3289fea25ea0d15971
  Resolved https://github.com/unslothai/unsloth.git to commit 4211cc01409e3ced4f7abebaf68e244193b46e2c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
import torch

In [3]:
max_seq_length = 512 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Phi-3-mini-4k-instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

Unsloth: You passed in `unsloth/Phi-3-mini-4k-instruct` and `load_in_4bit = True`.
We shall load `unsloth/Phi-3-mini-4k-instruct-bnb-4bit` for 4x faster loading.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


==((====))==  Unsloth: Fast Mistral patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r=64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha=16,
    lora_dropout=0, # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth", # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None # And LoftQ
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
alpaca_prompt = """<|user|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Question:
{}
### Response:
{}<|end|>
"""

EOS_TOKEN = tokenizer.eos_token
print(EOS_TOKEN)
def formatting_prompts_func(examples):
    inputs = examples["question"]
    outputs = examples["answer"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = alpaca_prompt.format(input, output)
        texts.append(text)
    return {"text": texts}

dataset = load_dataset("microsoft/orca-math-word-problems-200k", split="train[:1%]")
dataset = dataset.map(formatting_prompts_func, batched=True)

<|endoftext|>


In [6]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer([
    alpaca_prompt.format(
        "What is the answer of 241 - (-241) + 1?", # input
        "", # output - leave this blank for generation!
    )],
    return_tensors="pt"
).to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=256)

<s>  <|user|> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Question:
What is the answer of 241 - (-241) + 1?
### Response:
 <|end|> <|assistant|> The answer is 644.

Here's the step-by-step calculation:

1. Subtracting a negative number is the same as adding its positive counterpart. So, 241 - (-241) becomes 241 + 241, which equals 482.
2. Then, add 1 to the result: 482 + 1 equals 643.

However, there seems to be a mistake in the calculation. The correct calculation should be:

1. Subtracting a negative number is the same as adding its positive counterpart. So, 241 - (-241) becomes 241 + 241, which equals 482.
2. Then, add 1 to the result: 482 + 1 equals 483.

So, the correct answer is 483. <|end|> <|assistant|> The correct calculation is as follows:

1. Subtracting a negative number is the same as adding its positive counterpart. So, 241 - (-241) becomes 241 + 


In [7]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args = TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        num_train_epochs=1,
        warmup_steps=10,
        max_steps=100,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=3407,
        output_dir="outputs"
    )
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [8]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.992 GB of memory reserved.


In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 100
 "-____-"     Number of trainable parameters = 119,537,664


Step,Training Loss
10,2.293700
20,2.500700
30,2.430200
40,2.315800
50,2.326600
60,2.398700
70,2.328900
80,2.317500
90,2.373200
100,2.258400


In [10]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer([
    alpaca_prompt.format(
        "What is the answer of 241 - (-241) + 1?", # input
        "", # output - leave this blank for generation!
    )],
    return_tensors="pt"
).to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=256)

<s>  <|user|> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Question:
What is the answer of 241 - (-241) + 1?
### Response:
 <|end|> <|assistant|> The answer is 644.

Here's the step-by-step calculation:

1. Subtracting a negative number is the same as adding its positive counterpart. So, 241 - (-241) becomes 241 + 241, which equals 482.
2. Then, add 1 to the result: 482 + 1 equals 643.

However, there seems to be a mistake in the calculation. The correct calculation should be:

1. Subtracting a negative number is the same as adding its positive counterpart. So, 241 - (-241) becomes 241 + 241, which equals 482.
2. Then, add 1 to the result: 482 + 1 equals 483.

So, the correct answer is 483. <|end|> <|assistant|> The correct calculation is as follows:

1. Subtracting a negative number is the same as adding its positive counterpart. So, 241 - (-241) becomes 241 + 
